# Análisis exploratorio automatizado utilizando YData-Profiler

In [11]:
#LIBRERIAS
from ydata_profiling import ProfileReport
import pandas as pd

In [12]:
#LECTURA DE DATOS
ruta_csv = '../data/processed/unificado/poligonos_unificados_completo.csv'

delitos = pd.read_csv(ruta_csv)

In [13]:
delitos.head()

,cve_ent,cve_mun,cve_loc,nom_loc,CVE_COL,COLONIA,CP,otros_cp,total_incidentes,incidentes_alta,...,viviendas_totales,escolaridad_años_prom,pctj_menores18,pctj_hombres,pctj_mujeres,tasa_incidentes_per_1k,tasa_alta_severidad_per_1k,score_severidad,area_km2,densidad_poblacional
0,26,30,1,Hermosillo,2603000016735,ALTA FIRENZE NORTE RESIDENCIAL,83104.0,NaN,1,1,...,68.0,14.9,28.6%,49.4%,50.6%,12.987013,12.987013,3.000000,0.037555,2050.340413
1,26,30,1,Hermosillo,2603000011785,JORGE VALDEZ MUÃOZ,83104.0,NaN,3066,1404,...,366.0,9.4,35.7%,49.9%,50.1%,2836.262720,1298.797410,2.199609,0.108585,9955.346547
2,26,30,1,Hermosillo,2603000016335,VILLA VERDE CERRADA SAN VICENTE,83118.0,NaN,0,0,...,261.0,9.5,40.4%,52.4%,47.6%,0.000000,0.000000,0.000000,0.060584,11801.826946
3,26,30,1,Hermosillo,2603000011480,VILLA VENTURA,83159.0,NaN,0,0,...,86.0,14.2,21.7%,46.5%,53.5%,0.000000,0.000000,0.000000,0.017695,6781.506720
4,26,30,1,Hermosillo,2603000011663,NUEVO HERMOSILLO,83296.0,NaN,43761,15415,...,4102.0,10.7,29.4%,49.1%,50.9%,3465.119962,1220.603373,2.022691,1.266794,9969.263119


In [9]:
delitos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cve_ent                     700 non-null    int64  
 1   cve_mun                     700 non-null    int64  
 2   cve_loc                     700 non-null    int64  
 3   nom_loc                     700 non-null    object 
 4   CVE_COL                     700 non-null    int64  
 5   COLONIA                     700 non-null    object 
 6   CP                          700 non-null    float64
 7   otros_cp                    0 non-null      float64
 8   total_incidentes            700 non-null    int64  
 9   incidentes_alta             700 non-null    int64  
 10  incidentes_media            700 non-null    int64  
 11  incidentes_baja             700 non-null    int64  
 12  categorias_dict             525 non-null    object 
 13  partes_dia_dict             525 non

In [14]:
col_categoricas = ['CVE_COL', 'COLONIA','CP']
delitos[col_categoricas] = delitos[col_categoricas].astype('category')

col_numericas = ['pctj_menores18', 'pctj_hombres', 'pctj_mujeres']
for columna in col_numericas:
    delitos[columna] = delitos[columna].str[:-1]
delitos[col_numericas] = delitos[col_numericas].astype('float')

In [15]:
delitos_profile = ProfileReport(delitos, 
                                title="Reporte de la base de delitos por colonia en Hermosillo")

In [16]:
#Dado el tamaño del reporte, no se puede visualizar directamente en la libreta, se guardará directamente en la carpeta de reportes para su análisis a profundidad
delitos_profile.to_file("../reports/1.0-EDA_YDataProfiler.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:00<00:00, 106.80it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Interpretación y pasos a tomar a partir del análisis EDA

A partir de lo observado, podemos observar que hay varias columnas que no aportan nada a nuestro reporte, ya que contienen un único valor en todas las filas, como:
* **cve_ent** - clave de entidad, 26 - única para Sonora
* **cve_mun** - clave de municipio, 30 - único para Hermosillo
* **otros_cp** - columna vacía
* **cve_loc** - 1 para todas las filas
* **nom_loc** - Hermosillo para todas las filas
* **incidentes_fin_semana** - vacío
* **incidentes_quincena** - vacío
* **dias_semana_dict** - 25% valores faltantes
* **categorias_dict** - 25% valores faltantes
* **partes_dia_dict** - 25% valores faltantes
* **fecha_inicio** - 25% valores faltantes
* **fecha_fin** - 25% valores faltantes

Se correrá un nuevo EDA removiendo las columnas que no ofrecen información relevante:

In [17]:
#Remover categorias no relevantes
delitos_limpio = delitos.drop(['cve_ent',
                               'cve_mun',
                               'otros_cp', 
                               'cve_loc', 
                               'nom_loc', 
                               'incidentes_fin_semana', 
                               'incidentes_quincena', 
                               'dias_semana_dict', 
                               'categorias_dict',
                              'partes_dia_dict'], 
                              axis=1)

In [18]:
#Creacion de reporte
delitos_limpio_profile = ProfileReport(delitos_limpio, 
                                title="Reporte 2 de la base de delitos por colonia en Hermosillo")
delitos_limpio_profile.to_file("../reports/2.0-EDA_YDataProfiler.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:00<00:00, 781.92it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Interpretación
A partir del heatmap de correlación, podemos notar las siguientes observaciones obvias: 
* Área (km2) tiene una alta correlación con viviendas totales y población total
* El área (km2) y la densidad poblacional están correlacionadas negativamente, a mayor área, menor densidad
* El total de incidentes está fuertemente relacionado con el número de incidentes de alta, media y baja severidad, ya que estas columnas lo componen.
* El porcentaje de hombres y mujeres está negativamente correlacionado, ya que conforman la población total, a mayor porcentaje de hombres, menor porcentaje de mujeres.
* El número de incidentes de alta severidad está fuertemente correlacionado al score de severidad y la tasa de alta severidad por cada 1000 habitantes.

Y las siguientes observaciones interesantes: 
* La escolaridad de años promedio y la tasa de incidentes por cada mil habitantes es negativa pero casi nula, lo que podría indicar que el nivel de educación promedio, por sí solo, no tiene una relación lineal fuerte con la tasa total de incidentes por cada 1,000 personas